# 00 — Problem context

## Objective
Define the business question, target KPI, channels, scope, and assumptions for a Marketing Mix Model (MMM) built from scratch.

---

## Business question

### Question
What is the incremental impact of each marketing channel on weekly purchases (conversions) over time?

### Intended decision (what this model will be used for)

#### Decision
Use this MMM to guide **cross-channel budget allocation** (how much to invest in each channel) and to run **scenario analysis** (what happens if spend changes).

#### Rationale
MMM works best for **strategic / portfolio-level** decisions because it uses aggregated time series data and estimates *average* effects over time.

#### Low-level allocation (within-channel)
This project aims to eventually support **low-level allocation** (e.g. within Paid Search or Paid Social), but that requires an additional layer beyond MMM:
- experimentation (geo-lift / incrementality tests) where possible
- within-channel models or rules (campaign/creative segmentation)
- operational constraints (pacing, min/max spend)

For this reason, the initial MMM is treated as a **top-down allocator** across channels, not a daily campaign optimizer.

---

## Target (KPI)

### Decision
Use **ALL_PURCHASES** as the primary KPI.

### Definition
Total number of purchases completed in a given time period.

- Unit: count (purchases)
- Raw granularity: daily
- Modeling granularity: weekly (daily aggregated to weekly)

### Rationale
- Purchase counts are stable and interpretable for MMM learning.
- Revenue-based KPIs can be explored later once the modeling pipeline is stable.

### What would change my mind
- If order value varies strongly due to promotions/pricing changes, a revenue KPI may be preferable.

---

## Channels (initial)

### Decision
Use **paid media spend** as marketing inputs (baseline MMM).

Channels available in the dataset:
- GOOGLE_PAID_SEARCH_SPEND
- GOOGLE_SHOPPING_SPEND
- GOOGLE_PMAX_SPEND
- GOOGLE_DISPLAY_SPEND
- GOOGLE_VIDEO_SPEND
- META_FACEBOOK_SPEND
- META_INSTAGRAM_SPEND
- META_OTHER_SPEND
- TIKTOK_SPEND

### Rationale
Spend is the standard MMM input and provides the cleanest baseline for incremental impact estimation.

### What we will exclude initially
- Clicks and impressions (kept for later exploration)
- Organic/direct click metrics as “channels” (may be used as controls later)

---

## Potential control variables

Controls represent non-marketing drivers of the KPI.

### Minimum set (always included)
- Trend (time index)
- Seasonality (week-of-year / month)

### Optional (if available)
- Promotions / discount periods
- Pricing changes
- Product launches
- Major external events / holidays

### Rationale
Controls reduce the risk of attributing non-marketing variation to marketing channels.

---

## Assumptions (initial)

### 1) Carryover effects (adstock)
**Assumption:** Marketing effects can persist over time.
- Spend this week may affect purchases in future weeks.

**Why it matters:** Without carryover, upper-funnel channels may be undervalued.

### 2) Diminishing returns (saturation)
**Assumption:** Returns diminish as spend increases.
- Extra spend has decreasing marginal impact.

**Why it matters:** Without saturation, the model may recommend unrealistic “put everything in one channel” allocations.

### 3) Linear relationship after transformations
**Assumption:** After applying adstock and saturation, a linear model can approximate the response.

**Why it matters:** It keeps the baseline interpretable while capturing core MMM dynamics.

### 4) Observational (correlation-based)
**Assumption:** The model estimates associations, not true causal effects.

**Why it matters:** MMM supports decisions but does not prove causal impact without additional evidence.

### 5) Parameter stability in the modeling window
**Assumption:** Effects are roughly stable over the modeled period.

**Why it matters:** If effects change drastically (algorithm/product/pricing changes), a single average effect can mislead.

---

## Risks & limitations

### Omitted variables bias
Unobserved factors (promotions, pricing, PR, stockouts) may bias attribution.

### Multicollinearity
Channels may move together, making it difficult to separate their effects reliably.

### Misspecification risk
Wrong adstock/saturation choices can distort channel contributions and scenario recommendations.

### Short-term shocks
Sudden events can create unexplained spikes/drops and mislead coefficient estimates.

### Sensitivity to time window and validation strategy
Model results can change depending on the selected time period and holdout approach.

---

## Definition of Done
- KPI, channels, and scope clearly defined
- Intended decisions (high-level and low-level roadmap) explicitly stated
- Assumptions and limitations written explicitly
- Base modeling table schema sketched

---

## Base modeling table (draft)

Expected schema (weekly aggregated):

- week
- target (ALL_PURCHASES)
- spend_* per channel
- controls (trend, seasonality, promo flags if available)

---

## Modeling approach
The modeling approach will be incremental:

1. Linear regression baseline (interpretability, sanity checks)
2. Add temporal controls (trend / seasonality)
3. Introduce adstock transformations (carryover effects)
4. Introduce saturation effects (diminishing returns)
5. Explore semi-parametric models (e.g. GAM)
6. Optional Bayesian formulation for uncertainty quantification

This roadmap prioritizes interpretability and learning
over immediate predictive performance.


## Notes
No data loading in this notebook.
This notebook focuses on problem framing only.




# 00 — Contexto del problema

## Objetivo

Definir la pregunta de negocio, el KPI objetivo, los canales, el alcance y los supuestos para un modelo de Marketing Mix Modeling (MMM) construido desde cero.

---

## Pregunta de negocio

### Pregunta

¿Cuál es el impacto incremental de cada canal de marketing sobre las compras semanales (conversiones) a lo largo del tiempo?

### Decisión prevista (para qué se usará este modelo)

#### Decisión

Utilizar este MMM para guiar la **asignación de presupuesto entre canales** (cuánto invertir en cada canal) y para realizar **análisis de escenarios** (qué ocurre si cambia el gasto).

#### Justificación

El MMM funciona mejor para **decisiones estratégicas / de portfolio**, ya que utiliza series temporales agregadas y estima efectos *medios* a lo largo del tiempo.

#### Asignación de bajo nivel (dentro de canal)

Este proyecto tiene como objetivo, a largo plazo, apoyar la **asignación de presupuesto a bajo nivel** (por ejemplo, dentro de Paid Search o Paid Social), pero esto requiere una capa adicional más allá del MMM:

* experimentación (geo-lift / tests de incrementalidad) cuando sea posible
* modelos o reglas dentro de cada canal (segmentación por campaña o creativo)
* restricciones operativas (pacing, mínimos y máximos de inversión)

Por este motivo, el MMM inicial se trata como un **asignador top-down entre canales**, no como un optimizador diario de campañas.

---

## Target (KPI)

### Decisión

Usar **ALL_PURCHASES** como KPI principal.

### Definición

Número total de compras completadas en un periodo de tiempo determinado.

* Unidad: conteo (compras)
* Granularidad original: diaria
* Granularidad de modelado: semanal (agregación de diario a semanal)

### Justificación

* El número de compras es estable e interpretable para aprender MMM.
* Los KPIs basados en revenue se pueden explorar más adelante, una vez que el pipeline de modelado sea estable.

### Qué cambiaría esta decisión

* Si el valor medio del pedido varía fuertemente debido a promociones o cambios de precio, un KPI de revenue podría ser más adecuado.

---

## Canales (iniciales)

### Decisión

Usar **gasto en medios de pago** como inputs de marketing (MMM baseline).

Canales disponibles en el dataset:

* GOOGLE_PAID_SEARCH_SPEND
* GOOGLE_SHOPPING_SPEND
* GOOGLE_PMAX_SPEND
* GOOGLE_DISPLAY_SPEND
* GOOGLE_VIDEO_SPEND
* META_FACEBOOK_SPEND
* META_INSTAGRAM_SPEND
* META_OTHER_SPEND
* TIKTOK_SPEND

### Justificación

El gasto es el input estándar en MMM y proporciona el punto de partida más limpio para estimar impacto incremental.

### Qué se excluye inicialmente

* Clics e impresiones (se mantienen para exploraciones posteriores)
* Métricas de tráfico orgánico/directo como “canales” (podrán usarse como controles más adelante)

---

## Variables de control potenciales

Las variables de control representan factores no directamente controlados por marketing que pueden influir en el KPI.

### Conjunto mínimo (siempre incluido)

* Tendencia (índice temporal)
* Estacionalidad (semana del año / mes)

### Opcionales (si están disponibles)

* Promociones / periodos de descuento
* Cambios de precio
* Lanzamientos de producto
* Eventos externos relevantes / festivos

### Justificación

Las variables de control reducen el riesgo de atribuir a marketing variaciones que no dependen del marketing.

---

## Supuestos (iniciales)

### 1) Efectos de arrastre (adstock)

**Supuesto:** Los efectos del marketing pueden persistir en el tiempo.

* El gasto de esta semana puede afectar a compras en semanas futuras.

**Por qué importa:** Sin arrastre, los canales de upper funnel pueden infravalorarse.

---

### 2) Rendimientos decrecientes (saturación)

**Supuesto:** Los retornos disminuyen a medida que aumenta el gasto.

* El gasto adicional tiene un impacto marginal decreciente.

**Por qué importa:** Sin saturación, el modelo puede recomendar asignaciones irreales (por ejemplo, invertir todo en un solo canal).

---

### 3) Relación lineal tras transformaciones

**Supuesto:** Tras aplicar adstock y saturación, una relación lineal puede aproximar la respuesta.

**Por qué importa:** Permite mantener un modelo interpretable capturando las dinámicas clave del MMM.

---

### 4) Modelo observacional (basado en correlación)

**Supuesto:** El modelo estima asociaciones, no efectos causales reales.

**Por qué importa:** El MMM apoya la toma de decisiones, pero no demuestra causalidad sin evidencia adicional.

---

### 5) Estabilidad de parámetros en la ventana de modelado

**Supuesto:** Los efectos son aproximadamente estables durante el periodo analizado.

**Por qué importa:** Si los efectos cambian drásticamente (algoritmos, producto, precios), un único efecto medio puede inducir a error.

---

## Riesgos y limitaciones

### Sesgo por variables omitidas

Factores no observados (promociones, precios, PR, roturas de stock) pueden sesgar la atribución.

### Multicolinealidad

Los canales pueden moverse conjuntamente, dificultando la separación fiable de sus efectos.

### Riesgo de mala especificación

Una elección incorrecta de adstock o saturación puede distorsionar las contribuciones y las recomendaciones de escenarios.

### Shocks de corto plazo

Eventos repentinos pueden generar picos o caídas no explicadas y sesgar los coeficientes.

### Sensibilidad a la ventana temporal y validación

Los resultados pueden variar según el periodo seleccionado y la estrategia de validación.

---

## Definición de “Done”

* KPI, canales y alcance claramente definidos
* Decisiones previstas (high-level y roadmap low-level) explícitas
* Supuestos y limitaciones documentados
* Esquema base de la tabla de modelado definido

---

## Tabla base de modelado (borrador)

Esquema esperado (agregado semanalmente):

* week
* target (ALL_PURCHASES)
* spend_* por canal
* controles (tendencia, estacionalidad, flags de promoción si están disponibles)

---

## Enfoque de modelado

El enfoque de modelado será incremental:

1. Regresión lineal baseline (interpretabilidad y sanity checks)
2. Añadir controles temporales (tendencia / estacionalidad)
3. Introducir transformaciones de adstock (efectos de arrastre)
4. Introducir efectos de saturación (rendimientos decrecientes)
5. Explorar modelos semi-paramétricos (p. ej. GAM)
6. Formulación bayesiana opcional para cuantificar incertidumbre

---

## Notas

No se cargan datos en este notebook.
Este notebook se centra únicamente en el planteamiento del problema.

